In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2 # THAY ĐỔI: EfficientNetB0 -> ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input # THÊM MỚI
from tensorflow.keras.models import Model # THAY ĐỔI: Sequential -> Model (cho Functional API)
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten # THÊM Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# --- Constants ---
PROCESSED_DATA_DIR = os.path.expanduser('~/Repos/cat-classification/data/processed')
TRAIN_DIR = os.path.join(PROCESSED_DATA_DIR, 'train')
VAL_DIR = os.path.join(PROCESSED_DATA_DIR, 'val')
TEST_DIR = os.path.join(PROCESSED_DATA_DIR, 'test')

# Model and training parameters
IMG_WIDTH, IMG_HEIGHT = 224, 224 # Kích thước ảnh đầu vào cho ResNet50V2
BATCH_SIZE = 32
EPOCHS = 50 # Số epochs ban đầu, có thể điều chỉnh
LEARNING_RATE = 1e-4

# Xác định số lớp (số giống mèo)
if not os.path.exists(TRAIN_DIR):
    raise FileNotFoundError(f"Thư mục huấn luyện không tìm thấy tại: {TRAIN_DIR}. Hãy chạy notebook data_splitting trước.")
    
num_classes = len([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
print(f"Tìm thấy {num_classes} lớp (giống mèo).")

if num_classes == 0:
    raise ValueError(f"Không tìm thấy thư mục con nào trong {TRAIN_DIR}. Kiểm tra lại dữ liệu đã xử lý.")

Tìm thấy 66 lớp (giống mèo).


In [12]:
# --- Data Generators ---
# ImageDataGenerator cho tập huấn luyện với tăng cường dữ liệu
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # THAY ĐỔI: Sử dụng preprocess_input của ResNetV2
    # rescale=1./255,              # LOẠI BỎ: preprocess_input đã xử lý chuẩn hóa
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ImageDataGenerator cho tập validation và test (chỉ chuẩn hóa, không tăng cường)
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # THAY ĐỔI

# Tạo generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical', # Vì chúng ta có nhiều lớp
    shuffle=True
)

validation_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Không cần shuffle tập validation
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Không cần shuffle tập test
)

# Kiểm tra xem generator có hoạt động không
try:
    sample_batch_train, sample_labels_train = next(train_generator)
    print(f"Hình dạng batch huấn luyện: {sample_batch_train.shape}")
    print(f"Hình dạng nhãn huấn luyện: {sample_labels_train.shape}")
    
    sample_batch_val, sample_labels_val = next(validation_generator)
    print(f"Hình dạng batch validation: {sample_batch_val.shape}")
    print(f"Hình dạng nhãn validation: {sample_labels_val.shape}")
except Exception as e:
    print(f"Lỗi khi tạo batch mẫu từ generator: {e}")
    print("Hãy kiểm tra lại đường dẫn TRAIN_DIR, VAL_DIR và cấu trúc thư mục con.")


Found 7898 images belonging to 66 classes.
Found 1693 images belonging to 66 classes.
Found 1693 images belonging to 66 classes.
Hình dạng batch huấn luyện: (32, 224, 224, 3)
Hình dạng nhãn huấn luyện: (32, 66)
Hình dạng batch validation: (32, 224, 224, 3)
Hình dạng nhãn validation: (32, 66)


In [13]:
# --- Model Definition (Transfer Learning with ResNet50V2) ---

# 1. Tải base model (ResNet50V2) đã được huấn luyện trước trên ImageNet
base_model = ResNet50V2(
    weights='imagenet',
    include_top=False, # Loại bỏ lớp phân loại cuối cùng
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
)

# 2. Đóng băng các trọng số của base model
base_model.trainable = False

# 3. Xây dựng mô hình mới bằng cách thêm các lớp tùy chỉnh lên trên base_model (Functional API)
# Lấy đầu ra của base_model
x = base_model.output

# Thêm các lớp tùy chỉnh
x = Flatten()(x) # Sử dụng Flatten thay vì GlobalAveragePooling2D như trong đoạn code mới của bạn
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
# Lớp đầu ra với num_classes (đã xác định ở ô đầu tiên) và softmax
predictions = Dense(num_classes, activation='softmax')(x)

# Tạo mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=predictions)

# In cấu trúc mô hình
model.summary()

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 74,979,394 (286.02 MB)

 Trainable params: 51,414,594 (196.13 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [14]:
# --- Model Compilation ---
optimizer = Adam(learning_rate=LEARNING_RATE) # LEARNING_RATE = 1e-4 từ ô đầu tiên

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy', # Giữ nguyên vì class_mode='categorical'
    metrics=['accuracy'] # Theo dõi độ chính xác ('acc' và 'accuracy' là như nhau)
)
print("Đã biên dịch mô hình.")

Đã biên dịch mô hình.


In [15]:
# --- Callbacks ---
# ModelCheckpoint: Lưu lại mô hình tốt nhất dựa trên val_accuracy
checkpoint_filepath = os.path.expanduser('~/Repos/cat-classification/models/cat_breed_classifier_best.keras')
os.makedirs(os.path.dirname(checkpoint_filepath), exist_ok=True) # Tạo thư mục models nếu chưa có

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, # Lưu toàn bộ mô hình
    monitor='val_accuracy',
    mode='max',
    save_best_only=True, # Chỉ lưu mô hình nếu val_accuracy cải thiện
    verbose=1
)

# EarlyStopping: Dừng huấn luyện sớm nếu không có cải thiện
early_stopping_callback = EarlyStopping(
    monitor='val_loss', # Theo dõi val_loss
    patience=10,        # Số epochs không cải thiện trước khi dừng
    verbose=1,
    restore_best_weights=True # Khôi phục trọng số từ epoch tốt nhất
)

# ReduceLROnPlateau: Giảm learning rate khi không có cải thiện
reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # learning_rate sẽ nhân với factor này
    patience=5,
    min_lr=1e-6, # Learning rate tối thiểu
    verbose=1
)

callbacks_list = [model_checkpoint_callback, early_stopping_callback, reduce_lr_callback]

In [16]:
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"TensorFlow has access to the following GPU(s): {gpu_devices}")
    # You can also enable device placement logging in TensorFlow
    # tf.debugging.set_log_device_placement(True)
    # to see where operations are being placed.
else:
    print("TensorFlow does not have access to a GPU. Training will run on the CPU.")

TensorFlow has access to the following GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
# --- Model Training ---
print("Bắt đầu huấn luyện mô hình...")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    steps_per_epoch=train_generator.samples // BATCH_SIZE, # Số batch mỗi epoch
    validation_steps=validation_generator.samples // BATCH_SIZE # Số batch cho validation
)

print("Hoàn tất huấn luyện mô hình (giai đoạn đầu - feature extraction).")


Bắt đầu huấn luyện mô hình...
Epoch 1/50


2025-05-09 18:36:50.205092: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.59GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


233/246 ━━━━━━━━━━━━━━━━━━━━ 3s 238ms/step - accuracy: 0.0669 - loss: 5.1709

2025-05-09 18:37:52.205390: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.58GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.0681 - loss: 5.1232
Epoch 1: val_accuracy improved from -inf to 0.17849, saving model to /home/maidang/Repos/cat-classification/models/cat_breed_classifier_best.keras
246/246 ━━━━━━━━━━━━━━━━━━━━ 88s 303ms/step - accuracy: 0.0682 - loss: 5.1197 - val_accuracy: 0.1785 - val_loss: 3.4396 - learning_rate: 1.0000e-04
Epoch 2/50
  1/246 ━━━━━━━━━━━━━━━━━━━━ 19s 79ms/step - accuracy: 0.0938 - loss: 4.4293
Epoch 2: val_accuracy did not improve from 0.17849
246/246 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0938 - loss: 4.4293 - val_accuracy: 0.1749 - val_loss: 3.4425 - learning_rate: 1.0000e-04
Epoch 3/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 0.1285 - loss: 3.7534
Epoch 3: val_accuracy improved from 0.17849 to 0.20913, saving model to /home/maidang/Repos/cat-classification/models/cat_breed_classifier_best.keras
246/246 ━━━━━━━━━━━━━━━━━━━━ 70s 283ms/step - accuracy: 0.1285 - loss: 3.7532 - val_accuracy: 0.2091 - va

KeyboardInterrupt: 